In [52]:
import json
import networkx as nx
import dynetx as dn
import re
import numpy as np

In [53]:
#function to read .json file of the input graph
def readPreferenceGraph(filePath) :
    with open("./preferenceGraph.json") as file :
        data = json.load(file)

    G = nx.DiGraph()

    for edge in data["edges"] :
        G.add_edge(edge["source"], edge["target"], comment=edge["comment"])
    
    return G

In [54]:
def createInteractionGraph(preferenceGraph, threshold, maxInteractions, maxTimestamp) :
    n = preferenceGraph.number_of_nodes()

    #create an empty Interaction Graph
    interactionGraph = dn.DynGraph()
    
    def calculateProbability(weight) :
        if re.match(r".*perceives.*prefers.*collaborate.*|.*perceives.*prefers.*socialize.*", weight) :
            return 0.7
        elif re.match(r".*perceives.*avoids.*collaborate.*|.*perceives.*avoids.*socialize.*", weight) :
            return 0.3
        elif re.match(r".*prefers.*collaborate.*|.*prefers.*socialize.*", weight) :
            return 1
        else :
            return 0
        
    def analyzeQuiz() :        
        #dictionary {edge : probability of existing in the Preference Graph}
        edgesProbabilities = {}

        #examine each eage in Preference Graph
        #calculate probability. How strong is the connection between people?
        for u, v in preferenceGraph.edges() :
            if preferenceGraph.has_edge(v, u) :
            #(u,v) and (v,u) are checked together once
                if (v,u) in edgesProbabilities :
                    #(v,u) is already checked, so neglect (u,v)
                    continue 
                else :
                    #(u,v), (v,u) pair has not been checked
                    edgesProbabilities[(u,v)] = 0.5*(calculateProbability(preferenceGraph[u][v]["comment"]) + calculateProbability(preferenceGraph[v][u]["comment"]))
            else :
                edgesProbabilities[(u,v)] = calculateProbability(preferenceGraph[u][v]["comment"])

        #find possible peers for a student
        #step 1: find connecting links with prob < threshold
        possiblePeers = [[] for _ in range(n)]       
        for edge, prob in edgesProbabilities.items() :
            if prob < threshold :
                possiblePeers[edge[0]].append(edge[1])
                possiblePeers[edge[1]].append(edge[0])

        #step 2: keep all the other nodes as possible peers to interact with
        possiblePeers = list(map(lambda item : list(set(range(n)) - set(item)), possiblePeers))
        
        return possiblePeers
    
    
    #function to find total degree of a node from 0 to timestamp - 1
    def previousDegree(node, timestamp) :
        totalDegree = 0
        for time in range(1,timestamp) :
            totalDegree += interactionGraph.degree(node, t=time)
        
        return totalDegree
    
    def addEdgesAtTimestamp(timestamp) :
        numberOfInteractions = np.random.choice([2,3,4])

        for node in range(n) :
            #find for a node how many additional interactions could we add with respect to maxInteractions parameter
            possibleInteractions = maxInteractions - previousDegree(node, timestamp=timestamp)
            if possibleInteractions < 0 :
                continue
            
            #choose randomly from possible peers
            for interaction in np.random.choice(possiblePeers[node], size=min(numberOfInteractions, possibleInteractions), replace=False) :
                interactionGraph.add_interaction(node, interaction, t=timestamp)


    activities = ["to share content, chat in a social network platform",
              "to participate in debate/group presentation & discussion",
              "to play games",
              "to study in group or work on collaborative projects",
              "to be volunteers",
              "to be teammates in a sports team"]


    possiblePeers = analyzeQuiz()
    
    interactions = []

    #interactions at timetsamp t = 0
    #interactions based on a scale-free network model with m=3
    G = nx.barabasi_albert_graph(n=preferenceGraph.number_of_nodes(), m=3)
    interactionGraph.add_interactions_from(G.edges(), t=0)
    d = {}
    for i,j in G.edges() :
        d[(i,j)] = np.random.choice(activities)
    interactions.append(d)

    #interactions at timestamps t >= 1
    for time in range(1, maxTimestamp+1) :
        addEdgesAtTimestamp(timestamp=time)
        d = {}
        for i,j,_ in interactionGraph.interactions(t=time) :
            d[(i,j)] = np.random.choice(activities)
        interactions.append(d)

    
    return interactionGraph, interactions




In [55]:
path = "./preferenceGraph.json"

options = {
    "threshold" : 0.4,
    "maxInteractions" : 30,
    "maxTimestamp" : 10
}

G = createInteractionGraph(preferenceGraph=readPreferenceGraph(path), **options)

In [56]:
print("Interactions added...")
for i in range(11) :
    print(f"at t = {i}: {G[0].number_of_interactions(t=i)}")

Interactions added...
at t = 0: 291
at t = 1: 296
at t = 2: 385
at t = 3: 293
at t = 4: 198
at t = 5: 275
at t = 6: 138
at t = 7: 29
at t = 8: 2
at t = 9: 1
at t = 10: 0


In [64]:
G[1][2]

{(0, 44): 'to share content, chat in a social network platform',
 (0, 55): 'to participate in debate/group presentation & discussion',
 (0, 68): 'to play games',
 (0, 23): 'to be volunteers',
 (0, 72): 'to study in group or work on collaborative projects',
 (1, 87): 'to be volunteers',
 (1, 65): 'to participate in debate/group presentation & discussion',
 (1, 59): 'to participate in debate/group presentation & discussion',
 (1, 63): 'to participate in debate/group presentation & discussion',
 (1, 46): 'to be teammates in a sports team',
 (1, 70): 'to study in group or work on collaborative projects',
 (1, 82): 'to play games',
 (1, 83): 'to study in group or work on collaborative projects',
 (2, 99): 'to share content, chat in a social network platform',
 (2, 97): 'to participate in debate/group presentation & discussion',
 (2, 32): 'to play games',
 (2, 57): 'to participate in debate/group presentation & discussion',
 (2, 8): 'to play games',
 (2, 25): 'to share content, chat in a soc